In [17]:
# Jacob Craven, Kevin Endrijaitis, Braden Burgener, Adam Rich​
# CS 490 AA
# Final Project Code

import numpy as np
import math

In [22]:
def load_data(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
        weights = []
        for line in lines:
            weights.append(line.strip())
        file.close()
    for i in range(len(weights)):
        weights[i] = math.floor(float(weights[i]))
        if weights[i] <= 0:
            weights[i] = 1
    return weights

def knapsack(packages, capacity, package_limit):
    num_rows = len(packages) + 1
    num_cols = capacity + 1
    matrix = np.zeros((num_rows, num_cols), dtype=int)
    packages.sort(reverse=True)

    for i in range(1, num_rows):
        for j in range(1, num_cols):
            # print(type(packages[i-1]), j)
            if int(packages[i-1]) <= j:
                matrix[i][j] = max(matrix[i-1][j], matrix[i-1][j-int(packages[i-1])] + packages[i-1])
            else:
                matrix[i][j] = matrix[i-1][j]
    current_weight = matrix[-1][-1]
    current_deliveries = []
    remaining_package_limit = package_limit
    remaining_capacity = capacity
    for i in range(num_rows - 1, 0, -1):
        if current_weight <= 0 or remaining_package_limit <= 0 or remaining_capacity <= 0:
            break
        if current_weight == matrix[i-1][remaining_capacity]:
            continue
        elif remaining_capacity < packages[i-1]:
            break
        else:
            current_deliveries.append(packages[i-1])
            current_weight -= packages[i-1]
            remaining_package_limit -= 1
            remaining_capacity -= packages[i-1]
    current_deliveries.reverse()

    if len(current_deliveries) > package_limit:
        current_deliveries = current_deliveries[:package_limit]
    for package in current_deliveries:
        packages.remove(package)
    return current_deliveries

def greedy(packages, capacity, package_limit):
    current_weight = 0
    current_deliveries = []
    for package in packages:
        if current_weight + package <= capacity and len(current_deliveries) < package_limit:
            current_weight += package
            current_deliveries.append(package)
            packages.remove(package)
        else:
            break 
    return current_deliveries

In [19]:
def exp1():
    pass

def exp2():
    pass

def exp3():
    pass

In [20]:
weights = load_data("amazon_weights.txt")
print(len(weights))
ans = greedy(weights, 1000, 15)
print(ans)
print("Total weight: ", sum(ans))
print("Total packages: ", len(ans))
ans = knapsack(weights, 1000, 15)
print(ans)
print("Total weight: ", sum(ans))
print("Total packages: ", len(ans))

8552
[171, 12, 13, 9, 24, 33, 1, 46, 4, 1, 11, 21, 25, 3, 23]
Total weight:  397
Total packages:  15
[3, 1, 32, 12, 6, 12, 24, 59, 8, 140, 10, 14, 16, 5, 11]
Total weight:  353
Total packages:  15


In [23]:
weights = load_data("amazon_weights.txt")
print(len(weights))
ans = greedy(weights, 1000, 15)
print(ans)
print("Total weight: ", sum(ans))
print("Total packages: ", len(ans))
ans = knapsack(weights, 1000, 15)
print(ans)
print("Total weight: ", sum(ans))
print("Total packages: ", len(ans))

8552
[171, 12, 13, 9, 24, 33, 1, 46, 4, 1, 11, 21, 25, 3, 23]
Total weight:  397
Total packages:  15
[252, 250, 249, 249]
Total weight:  1000
Total packages:  4
